In [56]:
!pip install shiny scanpy matplotlib pandas numpy zipfile36


In [2]:
import zipfile
import os
import scanpy as sc
import matplotlib.pyplot as plt
import shutil
from shiny import App, ui, reactive, render


In [3]:
#function for unzipping the file

import zipfile
import os

def unzip_file(zip_path, extract_to):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted to: {extract_to}")


In [4]:
#import scanpy as sc
import matplotlib.pyplot as plt
import tempfile
import os

def preprocess_and_generate_umap(h5ad_file, color=None, n_neighbors=10, n_pcs=40, min_genes=200, max_genes=8000, min_cells=3, min_mean=0.0125, max_mean=3, min_disp=0.5, max_value=10):
    """
    Preprocesses single-cell data, performs UMAP dimensionality reduction, and generates a UMAP plot.

    Args:
        h5ad_file (str): Path to the .h5ad file.
        color (str, optional): Variable to color the UMAP plot by. Defaults to None.
        n_neighbors (int, optional): Number of neighbors for neighbor graph. Defaults to 10.
        n_pcs (int, optional): Number of principal components for neighbor graph. Defaults to 40.
        min_genes (int, optional): Minimum number of genes detected per cell. Defaults to 200.
        max_genes (int, optional): Maximum number of genes detected per cell. Defaults to 8000.
        min_cells (int, optional): Minimum number of cells expressing a gene. Defaults to 3.
        min_mean (float, optional): Minimum mean expression for highly variable gene selection. Defaults to 0.0125.
        max_mean (float, optional): Maximum mean expression for highly variable gene selection. Defaults to 3.
        min_disp (float, optional): Minimum dispersion for highly variable gene selection. Defaults to 0.5.
        max_value (float, optional): Maximum value after scaling. Defaults to 10.

    Returns:
        str: Path to the generated UMAP plot PNG file.
    """
    try:
        adata = sc.read(h5ad_file)
    except FileNotFoundError:
        print(f"Error: File not found at {h5ad_file}")
        return None

    sc.pp.calculate_qc_metrics(adata, percent_top=None, log1p=False, inplace=True)
    adata.raw = adata

    sc.pp.filter_cells(adata, min_genes=min_genes)
    sc.pp.filter_cells(adata, max_genes=max_genes)
    sc.pp.filter_genes(adata, min_cells=min_cells)

    sc.pp.normalize_total(adata, target_sum=1e4)
    sc.pp.log1p(adata)

    sc.pp.highly_variable_genes(adata, min_mean=min_mean, max_mean=max_mean, min_disp=min_disp)
    adata = adata[:, adata.var.highly_variable]

    sc.pp.scale(adata, max_value=max_value)

    sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=n_pcs)
    sc.tl.umap(adata)

    temp_file = tempfile.NamedTemporaryFile(suffix=".png", delete=False)
    sc.pl.umap(adata, color=color, show=False, save=temp_file.name)
    plt.close() # close plot to free memory.

    return temp_file.name


    


In [8]:
# UI for shiny

# Define the file upload UI
app_ui = ui.page_fluid(
    ui.input_file("file", "Upload a ZIP file", accept=".zip"),
    ui.output_ui("file_ui"),
    ui.output_plot("umap_plot")
)


In [12]:
def unzip_file(zip_path, extract_to):
    """Unzips a file to a specified directory."""
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_to)

# Define the server logic
def server(input, output, session):
    @reactive.Calc
    def uploaded_file():
        # Get the uploaded file path
        return input.file() if input.file() else None

    @output
    @render.ui
    def file_ui():
        if uploaded_file():
            # Print to check the file path after upload
            print(f"File uploaded: {uploaded_file()[0]['datapath']}")

            # Unzip the file
            unzip_file(uploaded_file()[0]['datapath'], 'extracted_files')
            print("File unzipped.")

            # Check if the .h5ad file is extracted
            h5ad_files = [f for f in os.listdir('extracted_files') if f.endswith('.h5ad')]
            if h5ad_files:
                print(f"Found .h5ad file: {h5ad_files[0]}")
                return ui.HTML(f"<p>Loaded {h5ad_files[0]}</p>")
            else:
                print("No .h5ad file found.")
                return ui.HTML("<p>No .h5ad file found in the ZIP.</p>")
        else:
            return ui.HTML("<p>No file uploaded</p>")

    @output
    @render.plot
    def umap_plot():
        if uploaded_file():
            # Generate UMAP plot
            h5ad_files = [f for f in os.listdir('extracted_files') if f.endswith('.h5ad')]
            if h5ad_files:
                h5ad_file = h5ad_files[0]
                umap_plot_path = preprocess_and_generate_umap(f'extracted_files/{h5ad_file}')
                if umap_plot_path:
                    # Load and return the plot from the saved file.
                    img = plt.imread(umap_plot_path)
                    plt.imshow(img)
                    plt.axis('off') #remove axis for cleaner look.
                    return plt.gcf()
                else:
                    return None # Return None if umap generation fails.
            else:
                return None
        else:
            return None

In [14]:
#the object creation 

# Create the app
app = App(app_ui, server)


In [16]:
pip install nest_asyncio


Note: you may need to restart the kernel to use updated packages.


In [19]:
from shiny import App, ui, render, reactive
import scanpy as sc
import matplotlib.pyplot as plt

# Define your UI layout
ui = ui.page_fluid(
    ui.input_file("file_upload", "Upload ZIP file", accept=".zip"),
    ui.output_plot("umap_plot")
)

# Define your server logic
def server(input, output, session):
    @reactive.Effect
    def on_file_upload():
        # File upload logic here
        if input.file_upload():
            zip_path = input.file_upload()["datapath"]
            # Process the file, unzip, load .h5ad file, generate UMAP
            # (add your unzipping and plotting logic here)
    
    @output()
    @render.plot
    def umap_plot():
        # Create and return your UMAP plot here
        # (add the code to generate the UMAP)
        return plt.figure()

# Create and run the app
app = App(ui=ui, server=server)

# Run the app
app.run()


RuntimeError: asyncio.run() cannot be called from a running event loop